In [1]:
!pip install -U -q httpx

## Import libraries

In [2]:
import os
import json
import pathlib
import httpx
from tqdm.notebook import tqdm
from threading import Thread
from queue import Queue

## Read the data from dataset
From example, dataset in path

```python
train_json = pahtlib.Path('../input/fathomnet-out-of-sample-detection/object_detection/train.json')
eval_json = pahtlib.Path('../input/fathomnet-out-of-sample-detection/object_detection/eval.json')
```

In [3]:
with train_json.open() as f:
    train_data = json.load(f)
    
with eval_json.open() as f:
    eval_data = json.load(f)

NameError: name 'train_json' is not defined

## Define threads and query

In [ ]:
from threading import Thread
from queue import Queue


def download_image(url, path):
    while True:
        try:
            r = httpx.get(url)
            break
        except:
            pass

    with open(path, 'wb') as f:
        f.write(r.content)


def download_images(images, path):
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    q = Queue()
    for image in images:
        q.put(image)

    bar = tqdm(total=q.qsize())

    def worker():
        while not q.empty():
            image = q.get()
            download_image(image['coco_url'], os.path.join(path, image['file_name']))
            q.task_done()
            bar.update()

    for _ in range(32):
        t = Thread(target=worker)
        t.daemon = True
        t.start()
    q.join()

## RUN!

In [ ]:
download_images(train_data['images'], 'images/train')
download_images(eval_data['images'], 'images/eval')